In [11]:
# Generating calibrated dataset (finalized)
# - Uses user-provided research table for Re_crit_low/high vs Mach (interpolated)
# - Mach range: 4.5 - 12
# - Shape: cone (with geometry columns)
# - Transition detection requires: N >= 7.5 AND Re_x within [Re_crit_low, Re_crit_high] (interpolated at local Mach)
# - Adds diagnostic columns: T_inf, rho_inf, mu_inf (viscosity at T_inf), U_inf, Tw_over_Tinf, Re_crit_low_used, Re_crit_high_used
# - Saves CSV to /mnt/data/generated_dataset_M4.5-12_cone_calibrated_v2.csv and shows a preview

import numpy as np, pandas as pd, os
from math import sqrt, tan, radians
import caas_jupyter_tools as tools

output_path = "/mnt/data/generated_dataset_M4.5-12_cone_calibrated_v2.csv"

# User-provided calibration table (clean copy)
cal_table = np.array([
    [25000, 3.5, 216.65, 0.0401, 1.45e-05, 1.1, 1032.648116, 2855806.17, 834279.7656, 1668559.531],
    [30000, 4.5, 228.65, 0.0184, 1.48e-05, 1.2, 1363.96455, 1695739.71, 1026979.795, 2053959.591],
    [35000, 5.5, 242.65, 0.00898, 1.55e-05, 1.25, 1717.346003, 994952.7163, 1229837.388, 2459674.775],
    [40000, 6.0, 250.35, 0.004, 1.60e-05, 1.3, 1902.961555, 475740.3888, 1315587.029, 2631174.058],
    [45000, 6.8, 264.15, 0.00193, 1.67e-05, 1.4, 2215.333865, 256023.6143, 1436762.233, 2873524.466],
    [50000, 7.5, 270.65, 0.00103, 1.70e-05, 1.5, 2473.262686, 149850.6215, 1530931.089, 3061862.178],
    [55000, 8.0, 260.15, 0.000571, 1.65e-05, 1.7, 2586.466563, 89507.41862, 1533929.978, 3067859.955],
    [60000, 8.5, 247.02, 0.00031, 1.58e-05, 2.0, 2677.872841, 52540.54308, 1502601.91, 3005203.82],
    [70000, 10.0, 214.65, 8.28e-05, 1.44e-05, 3.0, 2936.773229, 16886.44607, 1443375.673, 2886751.346],
    [80000, 12.0, 198.64, 1.85e-05, 1.32e-05, 4.0, 3390.155083, 4751.353715, 1500000.0, 3000000.0]
])

mach_points = cal_table[:,1]
Re_crit_low_points = cal_table[:,8]
Re_crit_high_points = cal_table[:,9]
T_inf_points = cal_table[:,2]
rho_points = cal_table[:,3]
mu_points = cal_table[:,4]
Tw_over_Tinf_points = cal_table[:,5]

# Interpolation function for critical Re low/high and also for T_inf/rho/mu for diagnostics
def interp_vals(mach):
    mach_clipped = np.clip(mach, mach_points.min(), mach_points.max())
    low = float(np.interp(mach_clipped, mach_points, Re_crit_low_points))
    high = float(np.interp(mach_clipped, mach_points, Re_crit_high_points))
    T_inf_est = float(np.interp(mach_clipped, mach_points, T_inf_points))
    rho_est = float(np.interp(mach_clipped, mach_points, rho_points))
    mu_est = float(np.interp(mach_clipped, mach_points, mu_points))
    TwTinf_est = float(np.interp(mach_clipped, mach_points, Tw_over_Tinf_points))
    if low > high:
        low, high = high, low
    return low, high, T_inf_est, rho_est, mu_est, TwTinf_est

# Physics helpers (ISA-like atmosphere, Sutherland viscosity, synthetic N surrogate)
R = 287.058
gamma = 1.4
mu0 = 1.716e-5
T0_ref = 273.15
S = 110.4

def atmosphere(h):
    if h < 11000:
        T = 288.15 - 0.0065 * h
        p = 101325.0 * (T / 288.15) ** 5.255877
    elif h < 20000:
        T = 216.65
        p11 = 101325.0 * (216.65 / 288.15) ** 5.255877
        p = p11 * np.exp(- (h - 11000) / 6341.97)
    else:
        T = 216.65
        rho0 = 0.08803
        scale_h = 7000.0
        rho = rho0 * np.exp(-(h - 20000)/scale_h)
        p = rho * R * T
        return T, p, rho
    rho = p / (R * T)
    return T, p, rho

def viscosity_sutherland(T):
    return mu0 * (T / T0_ref)**1.5 * (T0_ref + S) / (T + S)

def synthetic_N_from_physics(Re_x, M_e, T_w_over_T_e):
    Re_s = np.log10(max(Re_x, 1e3))
    M_s = np.clip(M_e, 0.1, 50)
    Tw_ratio = T_w_over_T_e
    N_mean = 0.5 * Re_s / (M_s + 1.0) - 1.0 * (Tw_ratio - 1.0)
    N = 0.8 * N_mean + 8.5
    N = N + np.random.normal(scale=0.45)
    return float(N)

def calculate_N_factor(h, U, T_inf, rho_inf, x_sensor_local, T_w_local):
    a = sqrt(gamma * R * T_inf)
    M_e = max(U / a, 1e-6)
    T_e = T_inf
    Pr = 0.72
    r = np.sqrt(Pr)
    T_r = T_e * (1.0 + (gamma - 1.0)/2.0 * r * M_e**2)
    T_star = 0.5 * (T_w_local + T_e) + 0.22 * (T_r - T_e)
    mu_star = viscosity_sutherland(max(T_star, 1.0))
    Re_x = rho_inf * U * max(1e-6, x_sensor_local) / max(mu_star, 1e-12)
    N = synthetic_N_from_physics(Re_x, M_e, T_w_local / T_e)
    return N, Re_x, M_e, mu_star

# Read transition parameters for roughness if available
trans_params_path = "/mnt/data/transition_parameters.csv"
rough_values = None
if os.path.exists(trans_params_path):
    try:
        trans_df = pd.read_csv(trans_params_path)
        for c in trans_df.columns:
            if c.lower() in ['roughness','k','k_s','ks','epsilon','eps','surface_roughness']:
                rough_values = trans_df[c].dropna().values
                break
    except Exception:
        rough_values = None

if rough_values is None or len(rough_values)==0:
    rough_min = 1e-7
    rough_max = 1e-3
else:
    rough_min = float(np.nanmin(rough_values))
    rough_max = float(np.nanmax(rough_values))

# Generation
n_samples = 10000
rng = np.random.default_rng(20250916)
rows = []

for i in range(n_samples):
    M0 = float(rng.uniform(4.5, 12.0))
    h0 = float(rng.uniform(25000.0, 100000.0))
    T0, p0, rho0 = atmosphere(h0)
    a0 = sqrt(gamma * R * T0)
    U0 = M0 * a0
    cone_half_angle = float(rng.uniform(5.0, 15.0))
    nose_radius = float(rng.uniform(0.005, 0.05))
    cone_slant_length = float(rng.uniform(0.5, 50.0))
    alpha_rad = radians(cone_half_angle)
    base_radius = cone_slant_length * tan(alpha_rad)
    if base_radius < nose_radius:
        base_radius = nose_radius * 1.5
    x_sensor_local = float(rng.uniform(0.01, min(10.0, cone_slant_length)))
    T_w_local = float(rng.uniform(280.0, 1200.0))
    m_vehicle = float(rng.uniform(200.0, 2000.0))
    A_ref = np.pi * base_radius**2

    # altitude sweep
    alt_grid = np.linspace(h0, 0.0, 300)
    # placeholders
    h_trans = None; M_trans = None; N_trans = None; Re_x_trans = None; mu_star_trans=None
    crit_low_used = None; crit_high_used = None
    for h in alt_grid:
        T_inf, p_inf, rho_inf = atmosphere(h)
        if rho_inf <= 0 or T_inf <= 0:
            continue
        N, Re_x, M_e, mu_star = calculate_N_factor(h, U0, T_inf, rho_inf, x_sensor_local, T_w_local)
        crit_low, crit_high, T_inf_est, rho_est, mu_est, TwTinf_est = interp_vals(M_e)
        # Transition condition: N >= 7.5 AND Re_x within critical bounds (interpolated)
        if N >= 7.5 and (Re_x >= crit_low and Re_x <= crit_high):
            h_trans = float(h)
            M_trans = float(M_e)
            N_trans = float(N)
            Re_x_trans = float(Re_x)
            mu_star_trans = float(mu_star)
            crit_low_used = crit_low
            crit_high_used = crit_high
            break

    if rough_values is None or len(rough_values)==0:
        roughness_sample = float(rng.uniform(rough_min, rough_max))
    else:
        roughness_sample = float(rng.choice(rough_values))

    # Also compute diagnostics at initial conditions (for user's provided U_inf etc)
    # U_inf at init altitude (based on M0 and local a0)
    U_inf_init = U0
    # Estimate T_inf/rho/mu at init Mach via interpolation (for diagnostic match to research table)
    crit_low_init, crit_high_init, T_inf_est_init, rho_est_init, mu_est_init, TwTinf_est_init = interp_vals(M0)

    row = {
        'sample_index': i,
        'shape': 'cone',
        'init_Mach': M0,
        'init_altitude_m': h0,
        'init_T_inf_K': T0,
        'init_rho_kg_m3': rho0,
        'init_mu_Pa_s': viscosity_sutherland(T0),
        'init_velocity_m_s': U_inf_init,
        'cone_half_angle_deg': cone_half_angle,
        'nose_radius_m': nose_radius,
        'cone_slant_length_m': cone_slant_length,
        'base_radius_m': base_radius,
        'x_sensor_m': x_sensor_local,
        'T_wall_K': T_w_local,
        'Tw_over_Tinf_init': T_w_local / max(T0,1.0),
        'mass_kg': m_vehicle,
        'A_ref_m2': A_ref,
        'roughness_m': roughness_sample,
        'transition_detected': bool(h_trans is not None),
        'transition_altitude_m': h_trans if h_trans is not None else np.nan,
        'transition_Mach': M_trans if M_trans is not None else np.nan,
        'N_at_transition': N_trans if N_trans is not None else np.nan,
        'Re_x_at_transition': Re_x_trans if Re_x_trans is not None else np.nan,
        'mu_star_at_transition': mu_star_trans if mu_star_trans is not None else np.nan,
        'Re_crit_low_used': crit_low_used if crit_low_used is not None else np.nan,
        'Re_crit_high_used': crit_high_used if crit_high_used is not None else np.nan
    }
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv(output_path, index=False)

tools.display_dataframe_to_user("Calibrated cone dataset preview", df.head(12))
{"n_rows": len(df), "output_path": output_path, "roughness_range_used_m": [rough_min, rough_max], "cal_table_mach_min_max": [float(mach_points.min()), float(mach_points.max())]}


,h_m,U_m_s,x_pos_m,nose_radius_m,T_wall_K,T_inf_K,p_inf_Pa,rho_inf,T_e_K,Re_x,...,N_factor,mack_amp,mack_freq_Hz,q_dot_W_m2,label,log10_Re_x,T_ratio,h_norm,U_norm,N_round
7216,21598.498858,2157.140312,1.248454,0.010,1101.449464,218.175525,4307.948158,0.068785,58.599909,2.399900e+07,...,2.391842e-06,0.041484,15881.722641,1.337209,0,7.380193,18.796095,0.431970,0.431428,0.000
9064,28225.788510,6782.102780,1.816837,0.020,1179.022799,224.751290,1562.082160,0.024212,11.456511,6.992356e+07,...,4.506961e-25,0.037183,8455.261864,0.408716,0,7.844624,102.912898,0.564516,1.356421,0.000
938,35399.402049,2518.711782,0.477400,0.050,994.217859,237.620634,542.769869,0.007957,55.338379,1.429739e+06,...,0.000000e+00,0.004178,24058.043891,0.238175,0,6.155257,17.966154,0.707988,0.503742,0.000
6396,68654.417787,3657.288953,0.511102,0.010,765.404534,223.267057,6.395632,0.000100,29.755679,4.443155e+04,...,0.000000e+00,0.000713,9509.156471,0.053275,0,4.647691,25.722973,1.373088,0.731458,0.000
10120,11897.116452,1286.492441,0.847855,0.010,699.096154,216.650000,19715.468041,0.317014,102.984102,3.911511e+07,...,8.206464e-04,0.015750,26591.171331,2.024302,0,7.592345,6.788389,0.237942,0.257298,0.001
9018,40522.695856,3377.995753,1.149751,0.020,406.527871,251.796426,267.654339,0.003703,40.996613,4.914228e+06,...,1.611835e-04,0.028908,12147.704902,0.282794,0,6.691455,9.916133,0.810454,0.675599,0.000
11082,19962.862689,3903.225347,1.686371,0.005,1039.312432,216.650000,5561.604214,0.089428,25.653719,1.177595e+08,...,6.125705e-11,0.087791,13266.570464,2.099818,0,8.070996,40.513129,0.399257,0.780645,0.000
9497,13762.139156,3892.385791,1.203388,0.010,1029.197340,216.650000,14709.550114,0.236522,25.760225,2.228474e+08,...,8.123872e-11,0.087074,21147.034624,2.417215,0,8.348007,39.952963,0.275243,0.778477,0.000
10416,20895.818323,4659.716500,0.904735,0.020,936.549879,217.477508,4806.334021,0.076989,19.726910,7.696805e+07,...,6.963281e-13,0.035143,20914.739063,0.902702,0,7.886310,47.475751,0.417916,0.931943,0.000
5920,72957.866702,4007.026359,0.229293,0.010,449.052932,212.386310,3.301062,0.000054,23.823037,1.947951e+04,...,0.000000e+00,0.000272,16655.759773,0.034961,0,4.289578,18.849525,1.459157,0.801405,0.000


In [9]:
critical_rey=pd.read_csv('Critical_Reynolds_Mach4.5(all) (1).csv')
critical_rey.sample(10)


,Pressure_Pa,Re_cr_Pressure,Temperature_K,Re_cr_Temperature,Compressibility_Z,Re_cr_Compressibility,WallTemp_K,Re_cr_WallTemp,Velocity_mps,Re_cr_Velocity,...,Viscosity_Pa_s,Re_cr_Viscosity,TurbulenceIntensity,Re_cr_Turbulence,SurfaceRoughness,Re_cr_Roughness,MachNumber,Re_cr_Mach,PressureGradient,Re_cr_PressureGradient
74,3863.636364,379039.3115,486.363636,236461.1133,1.098990,272977.9412,623.232323,197801.8455,1570.707071,325053.1986,...,0.000040,108270.67900,0.074747,255151.5152,0.007475,187878.7879,6.484848,268853.1064,989.898990,270303.0303
91,4636.363636,393115.8041,563.636364,226229.0426,1.167677,256920.4152,743.434343,184328.8354,1862.626263,353972.6487,...,0.000047,92369.15376,0.091919,244848.4848,0.009192,162121.2121,7.515152,257219.3797,1676.767677,249696.9697
48,2681.818182,352347.2382,368.181818,257056.9543,0.993939,301829.2683,439.393939,227482.8717,1124.242424,275002.6187,...,0.000029,146965.35460,0.048485,270909.0909,0.004848,227272.7273,4.909091,292270.3007,-60.606061,301818.1818
62,3318.181818,367675.7960,431.818182,245051.7349,1.050505,285576.9231,538.383838,209726.6024,1364.646465,302982.0653,...,0.000035,123247.60290,0.062626,262424.2424,0.006263,206060.6061,5.757576,278620.5278,505.050505,284848.4848
77,4000.000000,381677.8910,500.000000,234507.6732,1.111111,270000.0000,644.444444,195171.3802,1622.222222,330340.6490,...,0.000041,105078.42310,0.077778,253333.3333,0.007778,183333.3333,6.666667,266632.0714,1111.111111,266666.6667
24,1590.909091,317405.2298,259.090909,285635.6474,0.896970,334459.4595,269.696970,276528.5016,712.121212,218868.7743,...,0.000020,219317.52920,0.024242,285454.5455,0.002424,263636.3636,3.454545,324763.8905,-1030.303030,330909.0909
67,3545.454545,372579.8726,454.545455,241309.7464,1.070707,280188.6792,573.737374,204458.4531,1450.505051,312367.9498,...,0.000037,116531.11220,0.067677,259393.9394,0.006768,198484.8485,6.060606,274365.9371,707.070707,278787.8788
14,1136.363636,296748.4581,213.636364,302653.3643,0.856566,350235.8491,198.989899,312289.8826,540.404040,190662.8753,...,0.000016,275915.60120,0.014141,291515.1515,0.001414,278787.8788,2.848485,344112.8058,-1434.343434,343030.3030
35,2090.909091,335237.0696,309.090909,270908.2241,0.941414,318669.5279,347.474747,249873.9763,901.010101,246190.7177,...,0.000024,178941.08030,0.035354,278787.8788,0.003535,246969.6970,4.121212,308019.0083,-585.858586,317575.7576
56,3045.454545,361422.7033,404.545455,249895.1695,1.026263,292322.8346,495.959596,216726.3955,1261.616162,291320.1108,...,0.000033,132405.31950,0.056566,266060.6061,0.005657,215151.5152,5.393939,284127.4477,262.626263,292121.2121


In [10]:
table=pd.read_csv('table_csv(new).csv')
table.sample(5)

,Unnamed: 0,Altitude,Unnamed: 2,Temperature,Unnamed: 4,Unnamed: 5,Pressure,Unnamed: 7,Density,Unnamed: 9
33,-3400,-3398,310.25,37.1,310.25,"14,941",1.1207,1.4746,1.6777,1.3696
0,H (m),Z (m),T (K),t (°C),TM (K),P (mb),P (torr),P/Po,P (kg/m3),p/po
61,-2000,-1999,301.15,20,301.15,1.2777,9.5838,1.261,1.4781,1.2066
4,-4850,-4846,319.675,46.525,"3,19,675","17,486",1.3115,1.7257,1.9056,1.5556
75,-1300,-1300,296.6,23.45,"2,96,600",1.1794,8.8468,1.164,1.3853,1.1309
